<a href="https://colab.research.google.com/github/ysjgithub/dl/blob/master/resnet_crifa10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

plt.ion()   # interactive mode

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32,padding=4),
                       transforms.ToTensor()
                   ])), batch_size=128, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])), batch_size=128, shuffle=True, num_workers=4)



Extracting ./cifar-10-python.tar.gz to .


In [0]:
class resBlock(nn.Module):
  def __init__(self,input_channel,out_channel,stride=1):
    super(resBlock, self).__init__()
    self.left =nn.Sequential(
        nn.Conv2d(input_channel,out_channel,3,stride=stride,padding =1,bias=False),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel,out_channel,3,stride=1,padding=1,bias =False),
        nn.BatchNorm2d(out_channel)
    )
    self.shortcut = nn.Sequential()
    if stride != 1 or input_channel != out_channel:
      self.shortcut = nn.Sequential(
          nn.Conv2d(input_channel, out_channel, kernel_size=1,stride=stride, bias=False),
          nn.BatchNorm2d(out_channel)
          )

  def forward(self,x):
    out = self.left(x)
    out  = self.shortcut(x)
    out = F.relu(out)
    return out



In [0]:



class ResNet(nn.Module):
  def __init__(self,num_classes=10):
    super(ResNet,self).__init__()
    self.input_channel = 64
    self.downsample = nn.Sequential(
        nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True)
    )
    self.b1 = self.layers(resBlock,64,2,1)
    self.b2 = self.layers(resBlock,128,2,2)
    self.b3 = self.layers(resBlock,256,2,2)
    self.b4 = self.layers(resBlock,512,2,2)
    self.avg = nn.AvgPool2d(4)
    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512,num_classes)
    )


  def layers(self,block,output_channels,nums,stride):
    strides = [stride] + [1]*(nums-1)
    r  = []
    for stride in strides:
      r.append(block(self.input_channel,output_channels,stride))
      self.input_channel = output_channels
    return nn.Sequential(*r)

  def forward(self,x):
    out = self.downsample(x)
    out = self.b1(out)
    out = self.b2(out)
    out = self.b3(out)
    out = self.b4(out)
    out = self.avg(out)
    out = self.fc(out)
    return out

model = ResNet().to(device)
summary(model,(3,32,32))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
          resBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,

In [0]:
optimizer = optim.Adam(model.parameters(), weight_decay=0.001)

criterion = nn.CrossEntropyLoss().to(device)


In [0]:

def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)

def accuracy(data_loader):
    total_correct = 0
    for x, y in data_loader:
      x = x.to(device)
      y = one_hot(np.array(y.numpy()), 10)
      target_class = np.argmax(y, axis=1)
      predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
      total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(data_loader.dataset)
# accuracy(train_loader)


In [0]:


def train(epoch):
    for x, y in train_loader:
      optimizer.zero_grad()
      x, y = x.to(device), y.to(device)
      output = model(x)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
    train_accuracy_rate = accuracy(train_loader)
    test_accuracy_rate = accuracy(test_loader)
    print('epoch:{:04d}|loss:{:.4f}|'
      'train_accuracy:{:.4f}|test_accuracy:{:.4f}'.format(epoch,loss,train_accuracy_rate,test_accuracy_rate))
#   
# A simple test procedure to measure STN the performances on MNIST.
#
# train(1)

In [0]:

for i in range(200):
  train(i)

epoch:0000|loss:1.7502|train_accuracy:0.3717|test_accuracy:0.3894
epoch:0001|loss:1.7786|train_accuracy:0.4047|test_accuracy:0.4199


KeyboardInterrupt: ignored

In [0]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

